In [13]:
import pandas as pd
import pickle

import sys
sys.path.append('C:\\Users\\Tim\\PycharmProjects\\625.687\src') # your local repo here

path = "C:\\Users\\Tim\\Desktop\\MillionSongSubsetMillionSongSubset_dataframe_test.pkl" # your local data here
df = pickle.load(open(path, "rb"))
for c in sorted(df.columns):
    print(c)

analysis_sample_rate
analyzer_version
artist_7digitalid
artist_familiarity
artist_hotttnesss
artist_id
artist_latitude
artist_location
artist_longitude
artist_mbid
artist_mbtags
artist_mbtags_count
artist_name
artist_playmeid
artist_terms
artist_terms_freq
artist_terms_weight
audio_md5
bars_confidence
bars_start
beats_confidence
beats_start
danceability
duration
end_of_fade_in
energy
genre
idx_artist_mbtags
idx_artist_terms
idx_bars_confidence
idx_bars_start
idx_beats_confidence
idx_beats_start
idx_sections_confidence
idx_sections_start
idx_segments_confidence
idx_segments_loudness_max
idx_segments_loudness_max_time
idx_segments_loudness_start
idx_segments_pitches
idx_segments_start
idx_segments_timbre
idx_similar_artists
idx_tatums_confidence
idx_tatums_start
key
key_confidence
loudness
mode
mode_confidence
release
release_7digitalid
sections_confidence
sections_start
segments_confidence
segments_loudness_max
segments_loudness_max_time
segments_loudness_start
segments_pitches
segments

In [17]:
import msd_utils

# Set up dataframe 
subdf = df[["year", "segments_pitches"]]
subdf = subdf[subdf["year"] != 0]

# Featurize pitch data
pitch_features = []
for index, row in subdf.iterrows():
    row_feats = msd_utils.vectorize_segments_pitches(row["segments_pitches"])
    pitch_features.append(row_feats)
subdf["pitch_features"] = pitch_features

In [4]:
import Metrics

mf = Metrics.Metric() # metric factory
metrics_df = []

# Compute a square matrix of scores (every song against every song)
for index1, row1 in subdf.iterrows():
    metric_scores = []
    for index2, row2 in subdf.iterrows():
        score = mf.L_2_norm(row1["pitch_features"], row2["pitch_features"])
        metric_scores.append(score)
    metrics_df.append(metric_scores)

In [5]:
radius = 120 # picked a value after looking at the L2 outputs

# Group together elements which, for a given radius, have metric values less than some radius
# Using matrix indices to save groups; indices should stil align to position in subdf 

elements = set()
for i in range(len(metrics_df)):
    ball_centered_around_i = set()
    for j in range(len(metrics_df)):
        if metrics_df[i][j] < radius:
            ball_centered_around_i.add(j)
    ball = tuple(sorted(ball_centered_around_i))
    elements.add(ball)

In [6]:
len(elements)

14

In [7]:
def forms_topology(set_of_sets):
    """
    Hacky/lazy check to find out whether a given set of sets is likely to be a topology
    I was hoping that the elements generated by a given metric would metrize up into a topology, 
    but it doesn't look like that's (always) the case
    """
    x_in_topology = False
    closed_under_intersection = True
    closed_under_union = True
    
    X = set()
    for subset in set_of_sets:
        
        # Build X
        for element in subset:
            X.add(element)
        
        # Check intersection, union (lazy - only looks at pairs, not combos)
        for any_other_subset in set_of_sets:
            intersect = set([x for x in subset if x in any_other_subset])
            union = set(subset)
            for x in any_other_subset:
                union.add(x)
            intersect = tuple(sorted(intersect))
            union = tuple(sorted(union))
            if not intersect in set_of_sets and not intersect == ():
                closed_under_intersection = False
            elif not union in set_of_sets:
                closed_under_union = False
    
    # Check X in set
    if tuple(sorted(X)) in set_of_sets:
        x_in_topology = True
        
    is_topology = x_in_topology and closed_under_intersection and closed_under_union
    return is_topology

# Unfortunately, the elements don't form a topology. Not a problem, but it would've been cool
forms_topology(elements)

False

In [12]:
# Generate simplicial complexes from the data
from itertools import combinations

def generate_simplicial_complexes(set_of_sets):
    """
    Treat each set like it represents a simplicial complex, and generate inclusion power sets all the way down
    (simplicial complexes are closed under interection; if (abc), then (ab), (ac), and (bc) must be in the set)
    """
    combs = set()
    for simplicial_complex in set_of_sets:
        print(simplicial_complex, "\n")
        for i in range(1, 4):
            i_length_combos = combinations(simplicial_complex, i)
            for combo in i_length_combos:
                combs.add(tuple(sorted(combo)))
    return combs
generate_simplicial_complexes(elements)

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51) 

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 49, 50, 51) 

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 25, 26, 28, 29, 30, 31, 32, 33, 34, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51) 

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51) 

(0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 27, 28, 29, 30, 31, 33, 34, 35, 36, 38, 39, 40, 41, 42, 43, 44, 47, 48, 49, 50, 51) 

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22,

{(11, 24, 41),
 (10, 23, 33),
 (4, 27, 36),
 (3, 30, 44),
 (12, 13, 14),
 (6, 16, 51),
 (23, 27, 35),
 (1, 4, 28),
 (4, 30, 35),
 (3, 27, 43),
 (25, 30, 38),
 (23, 28, 38),
 (24, 29, 46),
 (25, 27, 45),
 (1, 20, 33),
 (0, 23, 41),
 (3, 11, 22),
 (15, 26, 49),
 (2, 20, 30),
 (6, 29, 46),
 (1, 17, 38),
 (5, 16, 38),
 (0, 18, 46),
 (10, 24, 49),
 (4, 19, 30),
 (13, 22, 28),
 (7, 31, 49),
 (8, 30, 33),
 (2, 9, 14),
 (6, 24, 41),
 (5, 21, 33),
 (29, 34, 48),
 (27, 28, 44),
 (8, 33, 40),
 (4, 32, 42),
 (8, 25, 36),
 (3, 6, 43),
 (2, 25, 35),
 (2, 33, 39),
 (0, 31, 51),
 (4, 6, 20),
 (1, 36, 47),
 (20, 26, 36),
 (2, 28, 36),
 (19, 31, 44),
 (33, 44, 47),
 (10, 16, 47),
 (9, 21, 39),
 (28, 38, 43),
 (20, 29, 33),
 (18, 22, 25),
 (30, 31, 50),
 (8, 22, 31),
 (34, 44, 48),
 (19, 24, 41),
 (11, 24, 50),
 (29, 42, 46),
 (10, 23, 42),
 (5, 39),
 (6, 7, 30),
 (9, 18, 34),
 (3, 30, 37),
 (23, 27, 42),
 (1, 4, 21),
 (5, 29, 50),
 (22, 28, 34),
 (0, 7, 29),
 (3, 27, 34),
 (2, 4, 42),
 (23, 28, 47),
 (2